__Assignment 1__

a) Extend the program to include the station number (not the station name) where the maximum/minimum temperature was measured.

For each question include the following data in the report and sort it as shown:
- year, station with the max, maxValue ORDER BY maxValue DESC 
- year, station with the min, minValue ORDER BY minValue DESC

In [1]:
# create the setup
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)

In [2]:
path_unifolder_temp_tiny = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/temperature-readings-tiny.csv"


In [3]:
# read and create rdd, this needs to be tranfsormed to data frame later on
temps= sc.textFile(path_unifolder_temp_tiny)
parts = temps.map(lambda l:l.split(";"))
tempReadings = parts.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0], time=p[2],  Temp=float(p[3]), quality=p[4]))


In [4]:
tempReadings.take(10)

[Row(Temp=6.8, date='2013-11-01', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.8, date='2013-11-01', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=5.8, date='2013-11-02', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=-1.1, date='2013-11-02', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=-0.2, date='2013-11-03', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.6, date='2013-11-03', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=6.5, date='2013-11-04', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.1, date='2013-11-04', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=4.2, date='2013-11-05', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.2, date='2013-11-05', quality='G', station='102170', time='18:00:00', year='2013')]

In [5]:
# create a data frame from rdd
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")
schemaTempReadings.show(10)

+----+----------+-------+-------+--------+----+
|Temp|      date|quality|station|    time|year|
+----+----------+-------+-------+--------+----+
| 6.8|2013-11-01|      G| 102170|06:00:00|2013|
| 3.8|2013-11-01|      G| 102170|18:00:00|2013|
| 5.8|2013-11-02|      G| 102170|06:00:00|2013|
|-1.1|2013-11-02|      G| 102170|18:00:00|2013|
|-0.2|2013-11-03|      G| 102170|06:00:00|2013|
| 5.6|2013-11-03|      G| 102170|18:00:00|2013|
| 6.5|2013-11-04|      G| 102170|06:00:00|2013|
| 5.1|2013-11-04|      G| 102170|18:00:00|2013|
| 4.2|2013-11-05|      G| 102170|06:00:00|2013|
| 3.2|2013-11-05|      G| 102170|18:00:00|2013|
+----+----------+-------+-------+--------+----+
only showing top 10 rows



In [7]:
# we want to know the max and min temp for a specifitc time interval 
# time interval >= 1950 & <= 2014
schemaTempReadings_filterd = schemaTempReadings.select(["Temp","year", "station"]).filter((schemaTempReadings["year"] >= 1950) &  
                                                                          (schemaTempReadings["year"] <= 2014))
schemaTempReadings_filterd.show(10)

+----+----+-------+
|Temp|year|station|
+----+----+-------+
| 6.8|2013| 102170|
| 3.8|2013| 102170|
| 5.8|2013| 102170|
|-1.1|2013| 102170|
|-0.2|2013| 102170|
| 5.6|2013| 102170|
| 6.5|2013| 102170|
| 5.1|2013| 102170|
| 4.2|2013| 102170|
| 3.2|2013| 102170|
+----+----+-------+
only showing top 10 rows



In [11]:
# code from Question 1 + groupBy for station
# max case
temp_year_station_max = schemaTempReadings_filterd.groupBy(["year", "station"]).agg(F.max("Temp").alias("Temp_max"))\
.sort("Temp_max", ascending = False)
temp_year_station_max.show(10)

+----+-------+--------+
|year|station|Temp_max|
+----+-------+--------+
|2014| 102170|    29.1|
|1960| 102190|    29.0|
|1959| 102190|    28.2|
|1958| 102190|    28.1|
|1956| 102190|    26.0|
|1957| 102190|    25.2|
|1955| 102190|    20.4|
|1961| 102190|    19.0|
|2013| 102170|    10.2|
+----+-------+--------+



In [10]:
# code from Question 1 + groupBy for station
# min case
temp_year_station_min = schemaTempReadings_filterd.groupBy(["year", "station"]).agg(F.min("Temp").alias("Temp_min"))\
.sort("Temp_min", ascending = True)
temp_year_station_min.show(10)

+----+-------+--------+
|year|station|Temp_min|
+----+-------+--------+
|1956| 102190|   -30.0|
|1960| 102190|   -28.3|
|1958| 102190|   -27.9|
|1955| 102190|   -26.2|
|2014| 102170|   -24.3|
|1961| 102190|   -23.5|
|1959| 102190|   -23.2|
|1957| 102190|   -19.9|
|2013| 102170|   -13.3|
+----+-------+--------+

